In [1]:
import numpy as np
import pandas as pd

import conv_model as conv
import tensorflow as tf

In [2]:
#### User input ####

## path of the train and test data
train_data = "../data/digit-recognizer/train.csv"
test_data = "../data/digit-recognizer/test.csv"

In [3]:
## read data into Dataframe
df_train = pd.read_csv(train_data)
df_train.info()

df_test = pd.read_csv(test_data)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


In [4]:
## get the first 400 rows of the data
#df_train = df_train.iloc[:400]
#print(len(df_train.columns))

In [5]:
## function to create input images
def create_input_images(df):
    im_arr = []
    for row in range(0, len(df.index)):
        ## create the image frame
        image = np.zeros((28, 28, 1))

        if(row%1000 == 0):
            print(row)
        
        ## loop over the pixel values
        for col in range(1, len(df.columns)):
            image[(col-1)//28][(col-1)%28] = np.array([df.iloc[row, col]])
    
        im_arr.append(image)

    return np.array(im_arr)

In [6]:
## create the arrays of input images
im_arr = create_input_images(df_train)
im_arr_test = create_input_images(df_test)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000


In [7]:
model1 = conv.ConvModel(conv_layer = 'Conv2D')

In [8]:
from tensorflow.keras.losses import CategoricalCrossentropy

In [9]:
## compile the model
model1.compile(optimizer = 'adam', loss = CategoricalCrossentropy(), metrics = ["accuracy"])

In [10]:
## create training input and one_hot encode
Y = df_train["label"].to_numpy()
Y_oh = tf.one_hot(Y, 10)

In [11]:
## fit the model (currently just testing the design, no final run yet)
model1.fit(im_arr, Y_oh, batch_size = 32, epochs = 10)

Epoch 1/10


2024-04-09 14:27:59.071215: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


(None, 3136)
(None, 3136)
1313/1313 [==============================] - 17s 13ms/step - loss: 0.2270 - accuracy: 0.9282
Epoch 2/10
1313/1313 [==============================] - 16s 12ms/step - loss: 0.0857 - accuracy: 0.9732
Epoch 3/10
1313/1313 [==============================] - 17s 13ms/step - loss: 0.0648 - accuracy: 0.9790
Epoch 4/10
1313/1313 [==============================] - 18s 14ms/step - loss: 0.0564 - accuracy: 0.9819
Epoch 5/10
1313/1313 [==============================] - 20s 15ms/step - loss: 0.0507 - accuracy: 0.9837
Epoch 6/10
1313/1313 [==============================] - 21s 16ms/step - loss: 0.0436 - accuracy: 0.9861
Epoch 7/10
1313/1313 [==============================] - 20s 15ms/step - loss: 0.0405 - accuracy: 0.9873
Epoch 8/10
1313/1313 [==============================] - 19s 14ms/step - loss: 0.0358 - accuracy: 0.9884
Epoch 9/10
1313/1313 [==============================] - 19s 14ms/step - loss: 0.0343 - accuracy: 0.9892
Epoch 10/10
1313/1313 [==========================

In [12]:
model1.summary()

Model: "conv_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 dropout (Dropout)           multiple                  0         
                                                                 
 conv_block (ConvBlock)      multiple                  664       
                                                                 
 db (DB)                     multiple                  1168      
                                                                 
 conv_block_2 (ConvBlock)    multiple                  3488      
                                                                 
 db_1 (DB)                   multiple                  4640      
                                                                 
 conv_block_4 (ConvBlock)    multiple                  4

In [13]:
## predict for the test data
preds = model1.predict(im_arr_test)

(32, 3136)
875/875 [==============================] - 8s 9ms/step


In [14]:
## add test predictions to the test dataframe
#print(preds.shape)
#print(len(np.argmax(preds, axis = 1)))
#print(len(np.arange(1, len(df_test.index)+1)))
#print(df_test.columns)

df_test["Label"] = np.argmax(preds, axis = 1)
df_test["ImageId"] = np.arange(1, len(df_test.index)+1)

In [16]:
## path of the test predictions
test_pred_path = "../data/digit-recognizer/test_preds3.csv"

## write results to csv file
#to_write = ["ImageId", "Label"]
#df_test.to_csv(test_pred_path, columns = to_write, index = False)